# Минимальные остовные деревья. Алгоритм Крускала.

**Задача.** Составить алгоритм Кускала на основе структуры Union_Find 

**Формат файла:**

- [число_вершин] [число_ребер]
- [одна_конечная_точка_ребра_1] [другая_конечная_точка_ребра_1] [стоимость_ребра_1]
- [одна_конечная_точка_ребра_2] [другая_конечная_точка_ребра_2] [стоимость_ребра_2]
- ...

**Замечание:** Стоимость ребер может быть отрицательной и не обязательно различна.

Сравнить временные затраты для данной задачи и для решения с помощью алгоритма Прима.
Привести и обосновать асимптотическую оценку временной сложности алгоритма.

## Чтение данных из файла.

In [10]:
def read_input(file_path):
    with open(file_path, 'r') as f:
        data = f.read().splitlines()
    n, m = map(int, data[0].split())
    edges = [tuple(map(int, line.split())) for line in data[1:]]
    
    # Коррекция нумерации вершин (если нужно)
    edges = [(u-1, v-1, cost) for u, v, cost in edges]
    return n, edges

n_test, edges_test = read_input("test.txt")
n_data, edges_data = read_input("data.txt")

print(n_test, len(edges_test))
print(edges_test)


6 10
[(0, 1, 6), (0, 3, 5), (0, 4, 4), (1, 3, 1), (1, 4, 2), (1, 2, 5), (1, 5, 3), (2, 5, 4), (3, 4, 2), (4, 5, 4)]


## Алгоритм Прима.

Наивный алгоритм Прима строит минимальное остовное дерево (MST) поэтапно, начиная с произвольной вершины и добавляя рёбра минимальной стоимости, которые соединяют уже построенную часть дерева с остальной частью графа.

**Описание алгоритма Прима:**
1. Инициализация: Выбираем произвольную начальную вершину (например, вершину 0). Отмечаем её как посещённую. Инициализируем список рёбер минимального остовного дерева (MST).
2. Выбор минимального ребра: На каждом шаге перебираем все рёбра графа. Находим минимальное ребро, которое соединяет уже посещённую вершину с непосещённой.
3. Добавление ребра в дерево: Добавляем выбранное ребро в MST. Отмечаем новую вершину как посещённую.

**Остановка.** Алгоритм завершает работу, когда все вершины графа включены в MST (количество рёбер в MST становится равно $n−1$, где $n$ — число вершин).

In [31]:
def prim(n, edges):

    visited = [False] * n
    mst = []
    mst_cost = 0
    visited[0] = True
    
    while len(mst) < n - 1:
        min_edge = None
        min_cost = float('inf')
        
        for u, v, cost in edges:
            if 0 <= u < n and 0 <= v < n:  # Проверка индексов
                if visited[u] and not visited[v] and cost < min_cost:
                    min_edge = (u, v, cost)
                    min_cost = cost
                elif visited[v] and not visited[u] and cost < min_cost:
                    min_edge = (v, u, cost)
                    min_cost = cost
        
        if min_edge:
            mst.append(min_edge)
            mst_cost += min_edge[2]
            visited[min_edge[1]] = True
    return mst_cost, mst

print(prim(n_test, edges_test)[1])


[(0, 4, 4), (4, 1, 2), (1, 3, 1), (1, 5, 3), (5, 2, 4)]


**Время и результат работы алгоритма Прима на тестовом наборе данных:**

In [14]:
import time

start_time = time.time()
prim_test = prim(n_test, edges_test)
end_time = time.time()

print(f'Стоимость MST графа на тестовом наборе данных с помощью алгоритма Прима: {prim_test[0]}')
print(f'Время работы алгоритма Прима на тестовом наборе данных: {end_time - start_time} секунд')

Стоимость MST графа на тестовом наборе данных с помощью алгоритма Прима: 14
Время работы алгоритма Прима на тестовом наборе данных: 0.0 секунд


**Время и результат работы алгоритма Прима на усложненном наборе данных:**

In [20]:
import time

start_time = time.time()
prim_data = prim(n_data, edges_data)
end_time = time.time()

print(f'Стоимость MST графа на усложненном наборе данных с помощью алгоритма Прима: {prim_data[0]}')
print(f'Время работы алгоритма Прима на усложненном наборе данных: {end_time - start_time} секунд')

Стоимость MST графа на усложненном наборе данных с помощью алгоритма Прима: -3612829
Время работы алгоритма Прима на усложненном наборе данных: 0.3373689651489258 секунд


## Наивный алгоритм Крускала.

Наивный подход к реализации алгоритма Крускала заключается в управлении компонентами связности вручную, без использования оптимизированной структуры данных (например, Union-Find). Мы будем хранить текущие компоненты как множества, обновляя их при добавлении рёбер в остовное дерево.

**Описание алгоритма Крускала (наивный подход):**
1. Инициализация: Сортируем рёбра графа по их весу. Каждая вершина изначально образует свою собственную компоненту связности.
2. Добавление рёбер: Перебираем рёбра в порядке возрастания их весов. Если конечные вершины ребра принадлежат разным компонентам, добавляем это ребро в остовное дерево.
3. Обновление компонент связности: При добавлении ребра объединяем две компоненты связности, содержащие его вершины.

**Остановка:** Алгоритм завершает работу, когда в остовное дерево включено $n−1$ рёбер ($n$ — число вершин).

In [34]:
def kruskal_naive(n, edges):
    # Сортируем рёбра по весу
    edges.sort(key=lambda x: x[2])
    
    # Список для хранения минимального остовного дерева
    mst = []
    mst_cost = 0
    
    # Изначально каждая вершина принадлежит своей компоненте
    components = {i: {i} for i in range(n)}
    
    for u, v, cost in edges:
        # Если вершины принадлежат разным компонентам, добавляем ребро
        if components[u] != components[v]:
            mst.append((u, v, cost))
            mst_cost += cost
            
            # Объединяем компоненты
            comp_u = components[u]
            comp_v = components[v]
            merged = comp_u.union(comp_v)
            
            for node in merged:
                components[node] = merged
                
            # Останавливаемся, если в дереве уже n-1 рёбер
            if len(mst) == n - 1:
                break
    
    return mst_cost, mst

**Время и результат работы алгоритма Крускала (наивный подход) на тестовом наборе данных:**

In [35]:
import time

start_time = time.time()
kruskal_test = kruskal_naive(n_test, edges_test)
end_time = time.time()

print(f'Стоимость MST графа на тестовом наборе данных с помощью алгоритма Крускала (наивный подход): {kruskal_test[0]}')
print(f'Время работы алгоритма Крускала (наивный подход) на тестовом наборе данных: {end_time - start_time} секунд')

Стоимость MST графа на тестовом наборе данных с помощью алгоритма Крускала (наивный подход): 14
Время работы алгоритма Крускала (наивный подход) на тестовом наборе данных: 0.0 секунд


**Время и результат работы алгоритма Крускала (наивный подход) на усложненном наборе данных:**

In [25]:
import time

start_time = time.time()
kruskal_data = kruskal_naive(n_data, edges_data)
end_time = time.time()

print(f'Стоимость MST графа на усложненном наборе данных с помощью алгоритма Крускала (наивный подход): {kruskal_data[0]}')
print(f'Время работы алгоритма Крускала (наивный подход) на усложненном наборе данных: {end_time - start_time} секунд')

Стоимость MST графа на усложненном наборе данных с помощью алгоритма Крускала (наивный подход): -3612829
Время работы алгоритма Крускала (наивный подход) на усложненном наборе данных: 0.01189565658569336 секунд


## Алгоритм Крускала на основе структуры Union_Find.

Алгоритм Крускала с использованием структуры данных Union-Find (или Disjoint Set Union, DSU) является более эффективным подходом для построения минимального остовного дерева (MST). Эта структура позволяет быстро объединять компоненты связности и проверять, принадлежат ли две вершины одной компоненте.

**Основная идея:**
1. Использовать Union-Find для управления компонентами связности.
2. Сортировать рёбра графа по весу.
3. Перебирать рёбра в порядке возрастания веса и добавлять их в остовное дерево, если вершины, которые они соединяют, принадлежат разным компонентам.

**Шаги алгоритма**
1. Инициализация: Создаём объект Union-Find для отслеживания компонент связности. Сортируем рёбра по весу.
2. Добавление рёбер: Перебираем рёбра в порядке возрастания их весов. Если вершины ребра принадлежат разным компонентам (проверка с помощью Union-Find), добавляем ребро в MST и объединяем компоненты.

**Остановка:** Алгоритм завершает работу, когда в MST включено $n−1$ рёбер ($n$ — число вершин).

In [21]:
class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n

    def find(self, node):
        if self.parent[node] != node:
            self.parent[node] = self.find(self.parent[node])
        return self.parent[node]

    def union(self, node1, node2):
        root1 = self.find(node1)
        root2 = self.find(node2)
        if root1 != root2:
            if self.rank[root1] > self.rank[root2]:
                self.parent[root2] = root1
            elif self.rank[root1] < self.rank[root2]:
                self.parent[root1] = root2
            else:
                self.parent[root2] = root1
                self.rank[root1] += 1


def kruskal(n, edges):
    edges.sort(key=lambda x: x[2])  # Сортировка рёбер по весу
    uf = UnionFind(n)
    mst = []
    mst_cost = 0

    for u, v, cost in edges:
        if uf.find(u) != uf.find(v):
            uf.union(u, v)
            mst.append((u, v, cost))
            mst_cost += cost

    return mst_cost, mst


**Время и результат работы алгоритма Крускала (на основе структуры Union_Find) на тестовом наборе данных:**

In [22]:
import time

start_time = time.time()
kruskal_test = kruskal(n_test, edges_test)
end_time = time.time()

print(f'Стоимость MST графа на тестовом наборе данных с помощью алгоритма Крускала (Union_Find): {kruskal_test[0]}')
print(f'Время работы алгоритма Крускала (Union_Find) на тестовом наборе данных: {end_time - start_time} секунд')

Стоимость MST графа на тестовом наборе данных с помощью алгоритма Крускала (Union_Find): 14
Время работы алгоритма Крускала (Union_Find) на тестовом наборе данных: 0.0 секунд


**Время и результат работы алгоритма Крускала (на основе структуры Union_Find) на усложненном наборе данных:**

In [30]:
import time

start_time = time.time()
kruskal_data = kruskal(n_data, edges_data)
end_time = time.time()

print(f'Стоимость MST графа на усложненном наборе данных с помощью алгоритма Крускала (Union_Find): {kruskal_data[0]}')
print(f'Время работы алгоритма Крускала (Union_Find) на усложненном наборе данных: {end_time - start_time} секунд')

Стоимость MST графа на усложненном наборе данных с помощью алгоритма Крускала (Union_Find): -3612829
Время работы алгоритма Крускала (Union_Find) на усложненном наборе данных: 0.0015425682067871094 секунд
